In [ ]:
# coding: utf-8

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow.contrib.slim as slim
import cPickle
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import  LabelEncoder
get_ipython().magic(u'matplotlib inline')

In [ ]:
## 获取数据，并且对特征进行标准化处理，同时将目标域数据进行切分，得到训练集和测试集
def get_data():
    with open('./feature/casia_988_feature.pkl') as f:
        casia = cPickle.load(f)
    with open('./feature/southeast_988_feature.pkl') as f:
        se = cPickle.load(f)
    return casia,se


casia,se = get_data()   ## se表示实验室自己录的数据

casia_tr = StandardScaler()
casia['data'] = casia_tr.fit_transform(casia['data'])

se_tr = StandardScaler()
se['data'] = se_tr.fit_transform(se['data'])

trX,teX,trY,teY = train_test_split(casia['data'],casia['label'],test_size=0.5)

In [ ]:
##构建自编码器并进行训练
input_dim = 988

X = tf.placeholder("float", [None, input_dim], name='input')

initializer = tf.contrib.layers.xavier_initializer()

def model(X):
    encoder_fc1 = slim.fully_connected(X,768,normalizer_fn=slim.batch_norm,activation_fn=tf.nn.relu,scope='encoder_fc1',weights_initializer=initializer)
    encoder_fc2 = slim.fully_connected(encoder_fc1,512,normalizer_fn=slim.batch_norm,activation_fn=tf.nn.relu,scope='encoder_fc2',weights_initializer=initializer)
    decoder_fc1 = slim.fully_connected(encoder_fc2,768,normalizer_fn=slim.batch_norm,activation_fn=tf.nn.relu,scope='decoder_fc1',weights_initializer=initializer)
    decoder_fc2 = slim.fully_connected(decoder_fc1,input_dim,normalizer_fn=None,activation_fn=None,scope='decoder_fc2',weights_initializer=initializer)
    return decoder_fc2,encoder_fc2

Z,enc_feature = model(X)  #enc_feature是利用自编码器对数据进行特征提取的接口

cost = tf.reduce_sum(tf.pow(X - Z, 2))  
train_op = tf.train.AdamOptimizer(0.0005).minimize(cost)
predict_op = Z

sess = tf.InteractiveSession()  #如果是用python而不是ipython的话，注意将InteractiveSession()替换为Session()
tf.global_variables_initializer().run()
batch = 30
for i in range(500):
    trX,trY = shuffle(trX,trY)
    for start, end in zip(range(0, len(trX), batch), range(batch, len(trX)+1, batch)):
        input_ = trX[start:end]
        sess.run(train_op, feed_dict={X: input_})
    print("epoch: {0}".format(i),"train_loss: {0}".format(sess.run(cost, feed_dict={X: trX})),
          "   test_loss: {0}".format(sess.run(cost, feed_dict={X: teX})))

In [ ]:
casia_train = sess.run(enc_feature,feed_dict={X:trX}) ##利用自编码器得到casia训练集的变换特征
casia_test = sess.run(enc_feature,feed_dict={X:teX})##利用自编码器得到casia测试集的变换特征
se_train = sess.run(enc_feature,feed_dict={X:se['data']})##利用自编码器得到实验室数据集的变换特征

tag_transform = LabelEncoder()   ##对标签进行变换
casia_train_y = tag_transform.fit_transform(trY)
casia_test_y = tag_transform.transform(teY)
se_y = tag_transform.transform(se['label'])
se_train,se_y = shuffle(se_train,se_y)

train_percent = 1  #控制casia训练集中用于训练分类器的数据的比例
train_x = np.vstack([casia_train[0:int(train_percent*len(casia_train))],se_train])
train_y = np.concatenate([casia_train_y[0:int(train_percent*len(casia_train_y))],se_y])

train = {'data':train_x,'label':train_y}
test = {'data':casia_test,'label':casia_test_y}

## 保存训练数据与测试数据
with open('./transfer_feature/train.pkl','w') as f:
    cPickle.dump(train,f)


with open('./transfer_feature/test.pkl','w') as f:
    cPickle.dump(test,f)